In [93]:
import sys
sys.path.append('../deepsurv')
import deepsurv

#from deepsurv_logger import DeepSurvLogger, TensorboardLogger
import utils
import viz
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import lasagne
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [24]:
aall_features = list(range(181)) 
clin_demo_comp = list(range(153,181)) 
clin_demo_cyto_gen_comp = list(range(1,181)) 
comp = list(range(162,181)) 
cyto_comp = list(range(84,153))+list(range(162,181)) 
cyto_gen_comp = list(range(1,153))+list(range(162,181)) 
eln_clin_demo_comp = [1]+list(range(153,181)) 
eln_cyto_comp = [0]+list(range(84,153))+list(range(162,181)) 
eln_cyto_gen_comp = list(range(153))+list(range(162,181)) 
eln_gen_comp = list(range(84))+list(range(162,181)) 
gen_comp = list(range(1,84))+list(range(162,181))
clin_comp = list(range(152,160))+list(range(162,181)) 
clin_cyto_comp = list(range(84,160))+list(range(162,181)) 
clin_gen_comp = list(range(1,84))+list(range(153,160))+list(range(162,181)) 
eln_clin_comp = [0]+list(range(153,160))+list(range(162,181))

#Without age
all_features_without_age = list(range(161))+list(range(162,181))
clin_demo_comp_without_age = list(range(153,161))+list(range(162,181))
clin_demo_cyto_gen_comp_without_age = list(range(1,161))+list(range(162,181))
eln_clin_demo_comp_without_age = [0]+list(range(153,161))+list(range(162,181))
            
            
            
#eln_clin_gen = list(range(84))+list(range(153,160))  
#eln_demo_gen = list(range(84))+[160,161] 
#eln_clin_demo_cyto_gen =list(range(162)) 
#eln_clin_demo_cyto = [0]+list(range(84,162))
#eln_clin_demo_gen = list(range(84))+list(range(153,162))
#eln_clin_demo = [0] + list(range(153,162))
#eln_clin = [0] + list(range(153,160))
#eln_cyto_gen = list(range(153))
#clin_demo_cyto_gen = list(range(1,162))
#clin_demo_cyto = list(range(84,162))
#clin_demo_gen = list(range(1,84))+list(range(153,162)) 
#clin_demo = list(range(153,162)) 
#cyto_gen = list(range(1,153))
#cyto = list(range(84,153))
#gen = list(range(1,84))
#clin_gen = list(range(1,84)) + list(range(153,160))  
#clin_cyto = list(range(84,160))  
#demo_gen = list(range(1,84)) + [160,161]
#demo_cyto = list(range(84,153)) + [160,161]

###Without age:

#eln_demo_gen_without_age = list(range(84)) + [160]
#eln_clin_demo_cyto_gen_without_age = list(range(161))
#eln_clin_demo_cyto_without_age = [0] + list(range(84,161))
#eln_clin_demo_gen_without_age = list(range(84)) + list(range(153,161))
#eln_clin_demo_without_age = [0] + list(range(153,161))
#clin_demo_cyto_gen_without_age = list(range(1,161))
#clin_demo_cyto_without_age = list(range(84,161))
#clin_demo_gen_without_age = list(range(1,84)) + list(range(153,161)) 
#clin_demo_without_age = list(range(153,161))
#demo_gen_without_age = list(range(1,84)) + [160]
#demo_cyto_without_age = list(range(84,153)) + [160]

In [28]:
train_df = pd.read_table('df_prognosis_features_ready_final_component.tsv').iloc[:,clin_demo_cyto_gen+[180,181]]
train,test = train_test_split(train_df,test_size=0.2,random_state=17)
train,val = train_test_split(train,test_size=0.2,random_state=17)

In [89]:
def dataframe_to_deepsurv_ds(df, event_col = 'os_status', time_col = 'os'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    #x=x_df
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }

In [30]:
### Min Max Scaler Normalization and good format of the data to feed the MLP network
scale = preprocessing.MinMaxScaler().fit(train)
df_train = pd.DataFrame(scale.transform(train.values), columns=train.columns, index=train.index)
df_val = pd.DataFrame(scale.transform(val.values), columns=val.columns, index=val.index)
df_test = pd.DataFrame(scale.transform(test.values), columns=test.columns, index=test.index)
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'os_status', time_col= 'os')
val_data = dataframe_to_deepsurv_ds(df_val, event_col = 'os_status', time_col= 'os')
test_data = dataframe_to_deepsurv_ds(df_test, event_col = 'os_status', time_col= 'os')

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
train_df = pd.read_table('prognosis_comp_elli.tsv')
train,test = train_test_split(train_df,test_size=0.2,random_state=17)
train,val = train_test_split(train,test_size=0.2,random_state=17)
### Min Max Scaler Normalization and good format of the data to feed the MLP network
scale = preprocessing.MinMaxScaler().fit(train)
df_train = pd.DataFrame(scale.transform(train.values), columns=train.columns, index=train.index)
df_val = pd.DataFrame(scale.transform(val.values), columns=val.columns, index=val.index)
df_test = pd.DataFrame(scale.transform(test.values), columns=test.columns, index=test.index)
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'os_status', time_col= 'os')
val_data = dataframe_to_deepsurv_ds(df_val, event_col = 'os_status', time_col= 'os')
test_data = dataframe_to_deepsurv_ds(df_test, event_col = 'os_status', time_col= 'os')
hid_layers=[[100,100,10],[200,500,10],[250,100,100],[500,100,100],[100,100,10],[200,500,10],[250,100,100],[500,100,100],[100,100,10],[200,500,10],[250,100,100],[500,100,100],[100,100,10],[200,500,10],[250,100,100],[500,100,100]]
l2_reg=[10,10,10,10,0.1,0.1,0.1,0.1,10,10,10,10,0.1,0.1,0.1,0.1]
lr=[1e-05,1e-05,1e-05,1e-05,1e-05,1e-05,1e-05,1e-05,1e-03,1e-03,1e-03,1e-03,1e-03,1e-03,1e-03,1e-03]
network=[None]*16
metrics=[None]*16
for i in range(16):

    hyperparams = {
        'L2_reg': l2_reg[i],
        'batch_norm': True,
        'dropout': 0.4,
        'hidden_layers_sizes': hid_layers[i],
        'learning_rate': lr[i],
        'lr_decay': 0.001,
        'momentum': 0.9,
        'n_in': train_data['x'].shape[1],
        'activation':'rectify'
        #'standardize': True
    }
    np.random.seed(17)
    network[i] = deepsurv.DeepSurv(**hyperparams)
    network[i].restored_update_params = None 
    logger = deepsurv.deepsurv_logger.TensorboardLogger(name = 'DeepSurvLogger', logdir = './logs/tensorboard/')
    np.random.seed(17)
    #update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. 
    update_fn=lasagne.updates.rmsprop
    n_epochs = 12000
    val_frequency=250
    patience=10000 # minimum number of epochs to train for
    metrics1[i] =  network[i].train(train_data,val_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn,patience=patience,validation_frequency=val_frequency)

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12,580 - Training step 0/12000     |                         | - loss: 29.5316 - ci: 0.4566
2019-06-10 06:56:12

In [97]:
for i in range(16):
    print (network[i].get_concordance_index(x=val_data['x'],t=val_data['t'],e=val_data['e']),network[i].get_concordance_index(x=test_data['x'],t=test_data['t'],e=test_data['e']),network[i].hyperparams)

0.6610389286515873 0.7064943405314281 {'n_in': 181, 'learning_rate': 1e-05, 'hidden_layers_sizes': [100, 100, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.4, 'batch_norm': True, 'standardize': False}
0.6661637447317889 0.7174761178453881 {'n_in': 181, 'learning_rate': 1e-05, 'hidden_layers_sizes': [200, 500, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.4, 'batch_norm': True, 'standardize': False}
0.6743060924212574 0.7198844023440636 {'n_in': 181, 'learning_rate': 1e-05, 'hidden_layers_sizes': [250, 100, 100], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.4, 'batch_norm': True, 'standardize': False}
0.6721863388114416 0.7198683471140724 {'n_in': 181, 'learning_rate': 1e-05, 'hidden_layers_sizes': [500, 100, 100], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10, 'L1_reg': 0.0, 'activation': 'rectify', 'dro

AttributeError: 'NoneType' object has no attribute 'get_concordance_index'

In [ ]:
train_df = pd.read_table('prognosis_comp_yanis.tsv')
train,test = train_test_split(train_df,test_size=0.2,random_state=17)
train,val = train_test_split(train,test_size=0.2,random_state=17)
### Min Max Scaler Normalization and good format of the data to feed the MLP network
scale = preprocessing.MinMaxScaler().fit(train)
df_train = pd.DataFrame(scale.transform(train.values), columns=train.columns, index=train.index)
df_val = pd.DataFrame(scale.transform(val.values), columns=val.columns, index=val.index)
df_test = pd.DataFrame(scale.transform(test.values), columns=test.columns, index=test.index)
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'os_status', time_col= 'os')
val_data = dataframe_to_deepsurv_ds(df_val, event_col = 'os_status', time_col= 'os')
test_data = dataframe_to_deepsurv_ds(df_test, event_col = 'os_status', time_col= 'os')
hid_layers=[[100,100,10],[200,500,10],[250,100,100],[500,100,100],[100,100,10],[200,500,10],[250,100,100],[500,100,100],[100,100,10],[200,500,10],[250,100,100],[500,100,100],[100,100,10],[200,500,10],[250,100,100],[500,100,100]]
l2_reg=[10,10,10,10,0.1,0.1,0.1,0.1,10,10,10,10,0.1,0.1,0.1,0.1]
lr=[1e-05,1e-05,1e-05,1e-05,1e-05,1e-05,1e-05,1e-05,1e-03,1e-03,1e-03,1e-03,1e-03,1e-03,1e-03,1e-03]
network1=[None]*16
metrics1=[None]*16
for i in range(16):

    hyperparams = {
        'L2_reg': l2_reg[i],
        'batch_norm': True,
        'dropout': 0.4,
        'hidden_layers_sizes': hid_layers[i],
        'learning_rate': lr[i],
        'lr_decay': 0.001,
        'momentum': 0.9,
        'n_in': train_data['x'].shape[1],
        'activation':'rectify'
        #'standardize': True
    }
    np.random.seed(17)
    network1[i] = deepsurv.DeepSurv(**hyperparams)
    network1[i].restored_update_params = None 
    logger = deepsurv.deepsurv_logger.TensorboardLogger(name = 'DeepSurvLogger', logdir = './logs/tensorboard/')
    np.random.seed(17)
    #update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. 
    update_fn=lasagne.updates.rmsprop
    n_epochs = 12000
    val_frequency=250
    patience=10000 # minimum number of epochs to train for
    metrics1[i] =  network1[i].train(train_data,val_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn,patience=patience,validation_frequency=val_frequency)

In [98]:
for i in range(16):
    print (network1[i].get_concordance_index(x=val_data['x'],t=val_data['t'],e=val_data['e']),network1[i].get_concordance_index(x=test_data['x'],t=test_data['t'],e=test_data['e']),network1[i].hyperparams)

ValueError: shapes (309,181) and (179,100) not aligned: 181 (dim 1) != 179 (dim 0)
Apply node that caused the error: dot(x, W)
Toposort index: 13
Inputs types: [TensorType(float32, matrix), TensorType(float64, matrix)]
Inputs shapes: [(309, 181), (179, 100)]
Inputs strides: [(4, 1236), (800, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{Composite{(i0 * (Composite{(((i0 - i1) * i2 * i3) + i4)}(i1, i2, i3, i4, i5) + Abs(Composite{(((i0 - i1) * i2 * i3) + i4)}(i1, i2, i3, i4, i5))))}}[(0, 1)](TensorConstant{(1, 1) of 0.5}, dot.0, InplaceDimShuffle{x,0}.0, InplaceDimShuffle{x,0}.0, InplaceDimShuffle{x,0}.0, InplaceDimShuffle{x,0}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/Users/taziy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3020, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "/Users/taziy/anaconda3/lib/python3.7/site-pac